# Información del Documento

Este documento forma parte de los recursos del Máster en Computación Cuántica en la Universidad Internacional de La Rioja (UNIR).

## Detalles del Cuaderno

- **Asignatura:** Computación Cuántica
- **Autores:** Albert Nieto Morales, Javier Hernanz Zajara
- **Fecha:** 2023-12-27

## Disponibilidad

Este cuaderno y otros recursos relacionados están disponibles en [el repositorio en GitHub](https://github.com/albertnieto/mucomcu04).

## Nota Importante

Este material educativo se enfoca en la Computación Cuántica y ha sido desarrollado como parte del programa de Máster en Computación Cuántica en UNIR. La información proporcionada en este documento puede estar sujeta a actualizaciones. Se recomienda verificar la fecha de la última actualización para asegurarse de contar con la información más reciente.


# Enunciado y librerías

### Introducción

En esta actividad, se busca desarrollar competencias clave en el manejo de conceptos fundamentales de computación cuántica. Los objetivos principales incluyen la utilización de puertas cuánticas básicas, la comprensión de la evolución del estado cuántico, la exploración de los estados de Bell y la implementación de un incrementador cuántico.

### Pautas de Elaboración

**Ejercicio 1: Evolución a Estados de Bell**

Se propone la implementación de cuatro circuitos cuánticos, cada uno evolucionando el estado del sistema a uno de los cuatro Estados de Bell. La descripción de la evolución se realiza tanto con notación de Dirac como en forma matricial. La verificación se lleva a cabo mediante la implementación en Qiskit Quantum Lab.

**Ejercicio 2: Suma Cuántica de Cúbits**

Se plantea la implementación de circuitos cuánticos de cuatro cúbits que suman los valores del 1 al 8 en un registro cuántico. La verificación de su correcto funcionamiento se realiza en Qiskit Quantum Lab.

**Ejercicio 3: Teleportación Cuántica**

Implementar en Python el algoritmo de teleportación cuántica para teleportar el estado  del cúbit de Alice al cúbit de Bob. El estado  viene determinado por los ángulos  grados y  grados, de la esfera de Bloch.

## Librerías

Los paquetes necesarios para realizar la actividad son:

- `qiskit`: La biblioteca principal de Qiskit.
- `qiskit_ibm_provider`: Proporciona acceso a los servicios en la nube de IBM Quantum.
- `qiskit-aer`: Proporciona acceso a simuladores cuánticos.

In [1]:
%%capture
%pip install qiskit
%pip install qiskit_ibm_provider
%pip install qiskit-aer

In [2]:
# Importing standard Qiskit libraries
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, QuantumCircuit, transpile, Aer
from qiskit_ibm_provider import IBMProvider
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.circuit.library import C3XGate

# Importing matplotlib
import matplotlib.pyplot as plt

# Importing Numpy, Cmath and math
import numpy as np
import os, math, cmath
from numpy import pi

# Other imports
from IPython.display import display, Math, Latex

Cargamos y actualizamos el entorno con las variables de entorno guardadas en `config.env`.

In [3]:
# Specify the path to your env file
env_file_path = 'config.env'

# Load environment variables from the file
os.environ.update(line.strip().split('=', 1) for line in open(env_file_path) if '=' in line and not line.startswith('#'))

# Load IBM Provider API KEY
IBMP_API_KEY = os.environ.get('IBMP_API_KEY')

Luego, extraemos la clave de API de IBM Quantum Provider y se guarda en la variable `IBMP_API_KEY`.

In [4]:
# Loading your IBM Quantum account(s)
IBMProvider.save_account(IBMP_API_KEY, overwrite=True)

# Run the quantum circuit on a statevector simulator backend
backend = Aer.get_backend('statevector_simulator')

# Ejercicio 1
Implementar cuatro circuitos cuánticos de forma que cada uno de ellos haga evolucionar el estado del sistema a cada uno de los cuatro estados de Bell. Describe la evolución del sistema paso a paso de dos formas: con notación de Dirac y en forma matricial. Finalmente, implementa los circuitos utilizando QISKIt Quantum Lab y verifica que los resultados son los esperados.

## Estados de Bell

En computación cuántica, los Estados de Bell son un conjunto de cuatro estados cuánticos maximamente entrelazados. Estos estados, también conocidos como "pares EPR" o "qubits entrelazados", son generados mediante una serie de operaciones cuánticas específicas. Los cuatro Estados de Bell se denotan como:

$$ |\Phi^+\rangle = \frac{1}{\sqrt 2}(|0\rangle_A \otimes |0\rangle_B+|1\rangle_A \otimes |1\rangle_B) = \frac{|00\rangle + |11\rangle}{\sqrt 2}  = \frac{1}{\sqrt 2} |00\rangle + \frac{1}{\sqrt 2} |11\rangle = \frac{1}{\sqrt 2} ( |00\rangle + |11\rangle ) $$

$$ |\Phi^-\rangle = \frac{1}{\sqrt 2}(|0\rangle_A \otimes |0\rangle_B - |1\rangle_A \otimes |1\rangle_B)  = \frac{|00\rangle - |11\rangle}{\sqrt 2}  = \frac{1}{\sqrt 2} |00\rangle - \frac{1}{\sqrt 2} |11\rangle = \frac{1}{\sqrt 2} ( |00\rangle - |11\rangle ) $$

$$ |\Psi^+ \rangle = \frac{1}{\sqrt 2}(|0\rangle_A \otimes |1\rangle_B+|1\rangle_A \otimes |0\rangle_B)  = \frac{|01\rangle + |10\rangle}{\sqrt 2}  = \frac{1}{\sqrt 2} |01\rangle + \frac{1}{\sqrt 2} |10\rangle = \frac{1}{\sqrt 2} ( |01\rangle + |10\rangle ) $$

$$ |\Psi^- \rangle = \frac{1}{\sqrt 2}(|0\rangle_A \otimes |1\rangle_B - |1\rangle_A \otimes |0\rangle_B) = \frac{|01\rangle - |10\rangle}{\sqrt 2} = \frac{1}{\sqrt 2} |01\rangle - \frac{1}{\sqrt 2} |10\rangle = \frac{1}{\sqrt 2} ( |01\rangle - |10\rangle ) $$

Para realizar este ejercicio, definamos los vectores de estado $|0⟩$ y $|1⟩$ con su forma matricial en NumPy.

In [5]:
sv_0 = np.array([1, 0])
sv_1 = np.array([0, 1])

## Funciones auxiliares

Para simplificar los pasos, se ha creado una función principal llamada `array_to_dirac_and_matrix_latex`. Esta función toma una matriz de NumPy que representa el estado del sistema y genera código LaTeX para visualizar tanto la representación matricial como la notación de Dirac del estado cuántico. Además, utiliza tres funciones auxiliares para realizar estas tareas:

- **`array_to_matrix_representation`**: Convierte un array unidimensional en una representación matricial en columna.

- **`array_to_dirac_notation`**: Convierte un array complejo que representa un estado cuántico en superposición a la notación de Dirac.

- **`find_symbolic_representation`**: Verifica si un valor numérico corresponde a una constante simbólica dentro de una tolerancia especificada. Si se encuentra una correspondencia, devuelve la representación simbólica como cadena (con un signo '-' si el valor es negativo), de lo contrario, devuelve el valor original.

### `is_symbolic_constant` Function:

**Purpose:**
The `is_symbolic_constant` function checks if an amplitude corresponds to a symbolic constant within a specified tolerance.

**Attributes:**
- `amplitude` (float): The amplitude to check.
- `symbolic_constants` (dict): A dictionary mapping numerical values to their symbolic representations.
- `tolerance` (float): Tolerance for comparing amplitudes with symbolic constants.

**Methods:**
None

**Example Usage:**
```python
symbol = is_symbolic_constant(my_amplitude)
```

In [6]:
def find_symbolic_representation(value, symbolic_constants={1/np.sqrt(2): '1/√2'}, tolerance=1e-10):
    """
    Check if the given numerical value corresponds to a symbolic constant within a specified tolerance.

    Parameters:
    - value (float): The numerical value to check.
    - symbolic_constants (dict): A dictionary mapping numerical values to their symbolic representations.
                                Defaults to {1/np.sqrt(2): '1/√2'}.
    - tolerance (float): Tolerance for comparing values with symbolic constants. Defaults to 1e-10.

    Returns:
    str or float: If a match is found, returns the symbolic representation as a string 
                  (prefixed with '-' if the value is negative); otherwise, returns the original value.
    """
    for constant, symbol in symbolic_constants.items():
        if np.isclose(abs(value), constant, atol=tolerance):
            return symbol if value >= 0 else '-' + symbol
    return value


### `array_to_dirac_notation` Function:

**Purpose:**
The `array_to_dirac_notation` function is designed to convert a complex-valued array representing a quantum state in superposition to Dirac notation.

**Attributes:**
- `array` (numpy.ndarray): The complex-valued array representing the quantum state in superposition.
- `tolerance` (float): Tolerance for considering amplitudes as negligible.

**Methods:**
None

**Example Usage:**
```python
dirac_notation = array_to_dirac_notation(my_quantum_state)


In [7]:
def array_to_dirac_notation(array, tolerance=1e-10):
    """
    Convert a complex-valued array representing a quantum state in superposition
    to Dirac notation.

    Parameters:
    - array (numpy.ndarray): The complex-valued array representing
      the quantum state in superposition.
    - tolerance (float): Tolerance for considering amplitudes as negligible.

    Returns:
    str: The Dirac notation representation of the quantum state.
    """
    # Ensure the statevector is normalized
    array = array / np.linalg.norm(array)

    # Get the number of qubits
    num_qubits = int(np.log2(len(array)))

    # Find indices where amplitude is not negligible
    non_zero_indices = np.where(np.abs(array) > tolerance)[0]

    # Generate Dirac notation terms
    terms = [
        (find_symbolic_representation(array[i]), format(i, f"0{num_qubits}b"))
        for i in non_zero_indices
    ]

    # Format Dirac notation
    dirac_notation = " + ".join([f"{amplitude}|{binary_rep}⟩" for amplitude, binary_rep in terms])

    return dirac_notation

### `array_to_matrix_representation` Function:

**Purpose:**
The `array_to_matrix_representation` function is designed to convert a one-dimensional array to a column matrix representation.

**Attributes:**
- `array` (numpy.ndarray): The one-dimensional array to be converted.

**Methods:**
None

**Example Usage:**
```python
matrix_rep = array_to_matrix_representation(my_array)


In [8]:
def array_to_matrix_representation(array):
    """
    Convert a one-dimensional array to a column matrix representation.

    Parameters:
    - array (numpy.ndarray): The one-dimensional array to be converted.

    Returns:
    numpy.ndarray: The column matrix representation of the input array.
    """
    # Replace symbolic constants with their representations
    matrix_representation = np.array([find_symbolic_representation(value) or value for value in array])

    # Return the column matrix representation
    return matrix_representation.reshape((len(matrix_representation), 1))

### `array_to_dirac_and_matrix_latex` Function:

**Purpose:**
The `array_to_dirac_and_matrix_latex` function is designed to generate LaTeX code for displaying both the matrix representation and Dirac notation of a quantum state.

**Attributes:**
- `array` (numpy.ndarray): The complex-valued array representing the quantum state.

**Methods:**
- `array_to_matrix_representation(array)`: Converts a one-dimensional array to a column matrix representation.
- `array_to_dirac_notation(array, tolerance=1e-10)`: Converts a complex-valued array representing a quantum state in superposition to Dirac notation.
- `is_symbolic_constant(amplitude, symbolic_constants={1/np.sqrt(2): '1/√2'}, tolerance=1e-10)`: Checks if an amplitude corresponds to a symbolic constant within a specified tolerance.

**Example Usage:**
```python
result = array_to_dirac_and_matrix_latex(my_quantum_state)


In [9]:
def array_to_dirac_and_matrix_latex(array):
    """
    Generate LaTeX code for displaying both the matrix representation and Dirac notation
    of a quantum state.

    Parameters:
    - array (numpy.ndarray): The complex-valued array representing the quantum state.

    Returns:

    Latex: A Latex object containing LaTeX code for displaying both representations.
    """
    matrix_representation = array_to_matrix_representation(array)
    latex = "Matrix representation\n\\begin{bmatrix}\n" + \
            "\\\\\n".join(map(str, matrix_representation.flatten())) + \
            "\n\\end{bmatrix}\n"
    latex += f'Dirac Notation:\n{array_to_dirac_notation(array)}'
    return Latex(latex)

## Estado de Bell $|\Phi^+\rangle$

El objetivo es llegar al estado 
$|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ partiendo del estado base $|00\rangle$

### Desarrollo usando notación de Dirac

1. Partimos de dos qubits $|A\rangle$ y $|B\rangle$ inicializados a $|0\rangle$, lo cual también ilustraremos como: $|0\rangle_A$ y $|0\rangle_B$
1. Aplicamos una puerta Hadamard sobre $|0\rangle_A$ tal que: $\langle H|A\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|0\rangle_A + |1\rangle_A)$
1. Puesto que tenemos 2 qubits en nuestro sistema, debemos representarlos como el producto tensorial: $\langle H|A\rangle \otimes |B\rangle$  Equivalente a $\frac{1}{\sqrt 2}(|00\rangle + |10\rangle)$
1. Por último, necesitamos realizar la operación CNOT en $|B\rangle$ controlado por $|A\rangle$, por lo que tendríamos: $ \langle \text{CNOT} | (\langle H|A\rangle \otimes |B\rangle)\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|00\rangle + |11\rangle)$

### Desarrollo utilizando Notación Matricial

1. **Definición de Qubits y Puertas Cuánticas:**
   - Definimos los qubits $|0\rangle_A = |0\rangle_B = \begin{bmatrix} 1 \\ 0 \end{bmatrix}$.
   - La puerta Hadamard se expresa como $H = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$.
   - La puerta CNOT se representa como $\text{CNOT} = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$.

2. **Aplicación de la Puerta Hadamard a $|A\rangle$:**
   - Aplicamos la puerta Hadamard a $|A\rangle$ de la siguiente manera:
     $$
     \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 1 \end{bmatrix}
     $$

3. **Cálculo del Estado del Sistema con $|B\rangle$:**
   - Calculamos el estado del sistema considerando $|B\rangle$ mediante el producto tensorial:
     $$
     \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 1 \end{bmatrix} \otimes \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 0 \\ 1 \\ 0 \end{bmatrix}
     $$

4. **Aplicación de la Puerta CNOT entre Qubits:**
   - Finalmente, aplicamos la puerta CNOT entre ambos qubits:
     $$
     \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix} \begin{bmatrix} 1 \\ 0 \\ 1 \\ 0 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 0 \\ 0 \\ 1 \end{bmatrix}
     $$
     Este resultado corresponde a la representación matricial del estado deseado.


Aprovechando las facilidades de Qiskit, el mismo circuito se representaría tal que:

In [10]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.h(qreg_q[0])
circuit.cx(qreg_q[0], qreg_q[1])

circuit.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
c: 1/══════════

### Desarrollo algorítmico

In [11]:
sv_b1 = np.kron(sv_0, sv_0)
array_to_dirac_and_matrix_latex(sv_b1)

<IPython.core.display.Latex object>

Añadimos una puerta Hadamard al primer qubit:

In [12]:
sv_b1 = (np.kron(sv_0, sv_0) + np.kron(sv_0, sv_1)) / np.sqrt(2)
array_to_dirac_and_matrix_latex(sv_b1)

<IPython.core.display.Latex object>

Finalmente añadimos una puerta CNOT, donde el qubit de control es el primer qubit, y el qubit objetivo es el segundo:

In [13]:
sv_b1 = (np.kron(sv_0, sv_0) + np.kron(sv_1, sv_1)) / np.sqrt(2)
array_to_dirac_and_matrix_latex(sv_b1)

<IPython.core.display.Latex object>

## Estado de Bell $|\Phi^-\rangle$

El objetivo es llegar al estado 
$|\Phi^-\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)$ partiendo del estado base $|00\rangle$

### Desarrollo usando notación de Dirac

1. Partimos de dos qubits $|A\rangle$ y $|B\rangle$ inicializados a $|0\rangle$, lo cual también ilustraremos como: $|0\rangle_A$ y $|0\rangle_B$
1. Aplicamos una puerta Pauli X (X) sobre $|0\rangle_A$ tal que: $\langle X|A\rangle$ Equivalente a $|1\rangle_A$
1. A continuación aplicamos una puerta Hadamard (H) sobre $|0\rangle_A$ tal que: $\langle H|X|A\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|0\rangle_A - |1\rangle_A)$
1. Puesto que tenemos 2 qubits en nuestro sistema, debemos representarlos como el producto tensorial: $\langle H|X|A\rangle \otimes |B\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|00\rangle - |10\rangle)$
1. Por último, necesitamos realizar la operación CNOT en $|B\rangle$ controlado por $|A\rangle$, por lo que tendríamos: $ \langle \text{CNOT} | (\langle H|X|A\rangle \otimes |B\rangle)\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|00\rangle - |11\rangle)$ 

### Desarrollo usando Notación Matricial

1. **Definición de Qubits y Puertas Cuánticas:**
   - Definimos los qubits $|0\rangle_A = |0\rangle_B = \begin{bmatrix} 1 \\ 0 \end{bmatrix}$.
   - La puerta Hadamard se expresa como $H = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$.
   - La puerta CNOT se representa como $\text{CNOT} = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$.
   - La puerta X (Pauli X) se define como $X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$.

2. **Aplicación de Puertas X y Hadamard a $|A\rangle$:**
   - Aplicamos la puerta X y, a continuación, la Hadamard sobre $|A\rangle$ de la siguiente manera:
     $$
     \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} 
     \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}
     \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ -1 \end{bmatrix}
     $$

3. **Cálculo del Estado del Sistema con $|B\rangle$:**
   - Calculamos el estado del sistema considerando $|B\rangle$ mediante el producto tensorial:
     $$
     \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ -1 \end{bmatrix} \otimes \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 0 \\ -1 \\ 0 \end{bmatrix}
     $$

4. **Aplicación de la Puerta CNOT entre Qubits:**
   - Finalmente, aplicamos la puerta CNOT entre ambos qubits:
     $$
     \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix} 
     \begin{bmatrix} 1 \\ 0 \\ -1 \\ 0 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 0 \\ 0 \\ -1 \end{bmatrix}
     $$
     Este resultado corresponde a la representación matricial del estado buscado.


Aprovechando las facilidades de Qiskit, el mismo circuito se representaría tal que:

In [14]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.x(qreg_q[0])
circuit.h(qreg_q[0])
circuit.cx(qreg_q[0], qreg_q[1])

circuit.draw()

┌───┐┌───┐     
q_0: ┤ X ├┤ H ├──■──
     └───┘└───┘┌─┴─┐
q_1: ──────────┤ X ├
               └───┘
c: 1/═══════════════

### Desarrollo algorítmico

In [15]:
sv_b2 = np.kron(sv_0, sv_0)
array_to_dirac_and_matrix_latex(sv_b2)

<IPython.core.display.Latex object>

Añadimos una puerta Hadamard al primer qubit:

In [16]:
sv_b2 = (np.kron(sv_0, sv_0) + np.kron(sv_0, sv_1)) / np.sqrt(2)
array_to_dirac_and_matrix_latex(sv_b2)

<IPython.core.display.Latex object>

Añadimos una puerta CNOT, donde el qubit de control es el primer qubit, y el qubit objetivo es el segundo:

In [17]:
sv_b2 = (np.kron(sv_0, sv_0) + np.kron(sv_1, sv_1)) / np.sqrt(2)
array_to_dirac_and_matrix_latex(sv_b2)

<IPython.core.display.Latex object>

Finalmente, añadimos una puerta Pauli-Z en el qubit 0 para pasar de $|+\rangle$ a $|-\rangle$.

In [18]:
sv_b2 = (np.kron(sv_0, sv_0) - np.kron(sv_1, sv_1)) / np.sqrt(2)
array_to_dirac_and_matrix_latex(sv_b2)

<IPython.core.display.Latex object>

## Estado de Bell $|\Psi^+\rangle$

El objetivo es llegar al estado 
$|\Psi^+\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$ partiendo del estado base $|00\rangle$

### Desarrollo usando notación de Dirac

1. Partimos de dos qubits $|A\rangle$ y $|B\rangle$ inicializados a $|0\rangle$, lo cual también ilustraremos como: $|0\rangle_A$ y $|0\rangle_B$
1. Aplicamos una puerta Pauli X (X) sobre $|0\rangle_B$ tal que: $\langle X|B\rangle$ Equivalente a $|1\rangle_B$
1. En paralelo aplicamos una puerta Hadamard (H) sobre $|0\rangle_A$ tal que: $\langle H|A\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|0\rangle_A + |1\rangle_A)$
1. Puesto que tenemos 2 qubits en nuestro sistema, debemos representarlos como el producto tensorial: $\langle H|A\rangle \otimes \langle X|B\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|01\rangle + |11\rangle)$
1. Por último, necesitamos realizar la operación CNOT en $|B\rangle$ controlado por $|A\rangle$, por lo que tendríamos: $ \langle \text{CNOT} | (\langle H|A\rangle \otimes \langle X|B\rangle)\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|01\rangle + |10\rangle)$ 

### Desarrollo usando Notación Matricial

1. **Definición de Qubits y Puertas Cuánticas:**
   - Definimos los qubits $|0\rangle_A = |0\rangle_B = \begin{bmatrix} 1 \\ 0 \end{bmatrix}$.
   - La puerta Hadamard se expresa como $H = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$.
   - La puerta CNOT se representa como $\text{CNOT} = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$.
   - La puerta X (Pauli X) se define como $X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$.

2. **Aplicación de la Puerta X sobre $|B\rangle$:**
   - Aplicamos la puerta X sobre $|B\rangle$ de la siguiente manera:
     $$
     \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}
     \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}
     $$

3. **Aplicación de la Puerta Hadamard sobre $|A\rangle$:**
   - Aplicamos la puerta Hadamard sobre $|A\rangle$ de la siguiente manera:
     $$
     \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} 
     \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 1 \end{bmatrix}
     $$

4. **Cálculo del Estado del Sistema con $|B\rangle$:**
   - Calculamos el estado del sistema considerando $|B\rangle$ mediante el producto tensorial:
     $$
     \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 1 \end{bmatrix} \otimes \begin{bmatrix} 0 \\ 1 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 0 \\ 1 \\ 0 \\ 1 \end{bmatrix}
     $$

5. **Aplicación de la Puerta CNOT entre Qubits:**
   - Finalmente, aplicamos la puerta CNOT entre ambos qubits:
     $$
     \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix} 
     \begin{bmatrix} 0 \\ 1 \\ 0 \\ 1 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 0 \\ 1 \\ 1 \\ 0 \end{bmatrix}
     $$
     Este resultado corresponde a la representación matricial del estado buscado.


Aprovechando las facilidades de Qiskit, el mismo circuito se representaría tal que:

In [19]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.h(qreg_q[0])
circuit.x(qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])

circuit.draw()

┌───┐     
q_0: ┤ H ├──■──
     ├───┤┌─┴─┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘
c: 1/══════════

### Desarrollo algorítmico

In [20]:
sv_b3 = np.kron(sv_0, sv_0)
array_to_dirac_and_matrix_latex(sv_b3)

<IPython.core.display.Latex object>

Añadimos una puerta Pauli-X al primer qubit:

In [21]:
sv_b3 = np.kron(sv_0, sv_1)
array_to_dirac_and_matrix_latex(sv_b3)

<IPython.core.display.Latex object>

Añadimos una puerta Hadamard al primer qubit:

In [22]:
sv_b3 = (np.kron(sv_0, sv_0) - np.kron(sv_0, sv_1)) / np.sqrt(2)
array_to_dirac_and_matrix_latex(sv_b3)

<IPython.core.display.Latex object>

Añadimos una puerta CNOT, donde el qubit de control es el primer qubit, y el qubit objetivo es el segundo:

In [23]:
sv_b3 = (np.kron(sv_0, sv_1) - np.kron(sv_1, sv_0)) / np.sqrt(2)
array_to_dirac_and_matrix_latex(sv_b3)

<IPython.core.display.Latex object>

Finalmente, añadimos una puerta Pauli-Z en el qubit 0 para pasar de $|+\rangle$ a $|-\rangle$.

In [24]:
sv_b3 = (np.kron(sv_0, sv_1) + np.kron(sv_1, sv_0)) / np.sqrt(2)
array_to_dirac_and_matrix_latex(sv_b3)

<IPython.core.display.Latex object>

## Estado de Bell $|\Psi^-\rangle$

El objetivo es llegar al estado 
$|\Psi^-\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$ partiendo del estado base $|00\rangle$

### Desarrollo usando notación de Dirac

1. Partimos de dos qubits $|A\rangle$ y $|B\rangle$ inicializados a $|0\rangle$, lo cual también ilustraremos como: $|0\rangle_A$ y $|0\rangle_B$
1. Aplicamos una puerta Pauli X (X) sobre $|0\rangle_B$ tal que: $\langle X|B\rangle$ Equivalente a $|1\rangle_B$
1. Aplicamos una puerta Pauli X (X) sobre $|0\rangle_A$ tal que: $\langle X|A\rangle$ Equivalente a $|1\rangle_A$
1. A continuación aplicamos una puerta Hadamard (H) en $|A\rangle$ tal que: $\langle H|X|A\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|0\rangle_A - |1\rangle_A)$
1. Puesto que tenemos 2 qubits en nuestro sistema, debemos representarlos como el producto tensorial: $\langle H|X|A\rangle \otimes \langle X|B\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|01\rangle - |11\rangle)$
1. Por último, necesitamos realizar la operación CNOT en $|B\rangle$ controlado por $|A\rangle$, por lo que tendríamos: $ \langle \text{CNOT} | (\langle H|X|A\rangle \otimes \langle X|B\rangle)\rangle$ Equivalente a $\frac{1}{\sqrt 2}(|01\rangle - |10\rangle)$ 

### Desarrollo usando Notación Matricial

1. **Definición de Qubits y Puertas Cuánticas:**
   - Definimos los qubits $|0\rangle_A = |0\rangle_B = \begin{bmatrix} 1 \\ 0 \end{bmatrix}$.
   - La puerta Hadamard se expresa como $H = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$.
   - La puerta CNOT se representa como $\text{CNOT} = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$.
   - La puerta X (Pauli X) se define como $X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$.

2. **Aplicación de la Puerta X sobre $|B\rangle$:**
   - Aplicamos la puerta X sobre $|B\rangle$ de la siguiente manera:
     $$
     \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}
     \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}
     $$

3. **Aplicación de Puertas X y Hadamard, en ese orden, sobre $|A\rangle$:**
   - Aplicamos las puertas X y Hadamard, en ese orden, sobre $|A\rangle$ de la siguiente manera:
     $$
     \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
     \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}
     \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ -1 \end{bmatrix}
     $$

4. **Cálculo del Estado del Sistema con $|B\rangle$:**
   - Calculamos el estado del sistema considerando $|B\rangle$ mediante el producto tensorial:
     $$
     \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ -1 \end{bmatrix} \otimes \begin{bmatrix} 0 \\ 1 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 0 \\ 1 \\ 0 \\ -1 \end{bmatrix}
     $$

5. **Aplicación de la Puerta CNOT entre Qubits:**
   - Finalmente, aplicamos la puerta CNOT entre ambos qubits:
     $$
     \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix} 
     \begin{bmatrix} 0 \\ 1 \\ 0 \\ -1 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 0 \\ 1 \\ -1 \\ 0 \end{bmatrix}
     $$
     Este resultado corresponde a la representación matricial del estado buscado.


Aprovechando las facilidades de Qiskit, el mismo circuito se representaría tal que:

In [25]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.x(qreg_q[0])
circuit.h(qreg_q[0])
circuit.x(qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])

circuit.draw()

┌───┐┌───┐     
q_0: ┤ X ├┤ H ├──■──
     ├───┤└───┘┌─┴─┐
q_1: ┤ X ├─────┤ X ├
     └───┘     └───┘
c: 1/═══════════════

### Desarrollo algorítmico

In [26]:
sv_b4 = np.kron(sv_0, sv_0)
array_to_dirac_and_matrix_latex(sv_b4)

<IPython.core.display.Latex object>

Añadimos una puerta Pauli-X al primer qubit:

In [27]:
sv_b4 = np.kron(sv_0, sv_1)
array_to_dirac_and_matrix_latex(sv_b4)

<IPython.core.display.Latex object>

Añadimos una puerta Hadamard al primer qubit:

In [28]:
sv_b4 = (np.kron(sv_0, sv_0) - np.kron(sv_0, sv_1)) / np.sqrt(2)
array_to_dirac_and_matrix_latex(sv_b4)

<IPython.core.display.Latex object>

Añadimos una puerta CNOT, donde el qubit de control es el primer qubit, y el qubit objetivo es el segundo:

In [29]:
sv_b3 = (np.kron(sv_0, sv_1) - np.kron(sv_1, sv_0)) / np.sqrt(2)
array_to_dirac_and_matrix_latex(sv_b3)

<IPython.core.display.Latex object>

## Implementaciones con Qiskit
A continuación, se implementan los circuitos descritos anteriormente y se verifican haciendo uso de Qiskit.

### Circuito y verificación para $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$

In [30]:
qc_b1 = QuantumCircuit(2, 2)
qc_b1.h(0)
qc_b1.cx(0, 1)

sv = backend.run(qc_b1).result().get_statevector()
sv.draw(output='latex', prefix = "|\Phi^+\\rangle = ")

<IPython.core.display.Latex object>

In [31]:
qc_b1.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
c: 2/══════════

### Circuito y verificación para $|\Phi^-\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)$

In [32]:
qc_b2 = QuantumCircuit(2, 2)
qc_b2.h(0)
qc_b2.cx(0, 1)
qc_b2.z(0)

sv = backend.run(qc_b2).result().get_statevector()
sv.draw(output='latex', prefix = "|\Phi^-\\rangle = ")

<IPython.core.display.Latex object>

In [33]:
qc_b2.draw()

┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ Z ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├─────
          └───┘     
c: 2/═══════════════

### Circuito y verificación para $|\Psi^+\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$

In [34]:
qc_b3 = QuantumCircuit(2, 2)
qc_b3.h(0)
qc_b3.cx(0, 1)
qc_b3.x(0)

sv = backend.run(qc_b3).result().get_statevector()
sv.draw(output='latex', prefix = "|\Psi^+\\rangle = ")

<IPython.core.display.Latex object>

In [35]:
qc_b3.draw()

┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ X ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├─────
          └───┘     
c: 2/═══════════════

### Circuito y verificación para $|\Psi^-\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$

In [36]:
qc_b4 = QuantumCircuit(2, 2)
qc_b4.h(0)
qc_b4.cx(0, 1)
qc_b4.x(0)
qc_b4.z(1)

sv = backend.run(qc_b4).result().get_statevector()
sv.draw(output='latex', prefix = "|\Psi^-\\rangle = ")

<IPython.core.display.Latex object>

In [37]:
qc_b4.draw()

┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ X ├
     └───┘┌─┴─┐├───┤
q_1: ─────┤ X ├┤ Z ├
          └───┘└───┘
c: 2/═══════════════

# Ejercicio 2
Implementar los circuitos de cuatro cúbits que suman al registro cuántico los valores 1, 2, 3, 4, 5, 6, 7 y 8 y verificar que funcionan correctamente. Utilizar QISKit Quantum lab.

In [38]:
def sv_latex_from_qc(qc, backend):
    sv = backend.run(qc).result().get_statevector()
    return sv.draw(output='latex')


Creamos un circuito cuántico de 4 qubits

In [39]:
qc_ej2 = QuantumCircuit(4, 4)
qc_ej2.x(0)
sv_latex_from_qc(qc_ej2, backend)

<IPython.core.display.Latex object>

In [40]:
qc_ej2 = QuantumCircuit(4, 4)
qc_ej2.x(1)
sv_latex_from_qc(qc_ej2, backend)

<IPython.core.display.Latex object>

In [41]:
qc_ej2 = QuantumCircuit(4, 4)
qc_ej2.x(0)
qc_ej2.x(1)
sv_latex_from_qc(qc_ej2, backend)

<IPython.core.display.Latex object>

In [42]:
qc_ej2 = QuantumCircuit(4, 4)
qc_ej2.x(2)
sv_latex_from_qc(qc_ej2, backend)

<IPython.core.display.Latex object>

In [43]:
qc_ej2 = QuantumCircuit(4, 4)
qc_ej2.x(0)
qc_ej2.x(2)
sv_latex_from_qc(qc_ej2, backend)

<IPython.core.display.Latex object>

In [44]:
qc_ej2 = QuantumCircuit(4, 4)
qc_ej2.x(0)
qc_ej2.x(2)
sv_latex_from_qc(qc_ej2, backend)

<IPython.core.display.Latex object>

In [45]:
qc_ej2 = QuantumCircuit(4, 4)
qc_ej2.x(0)
qc_ej2.x(1)
qc_ej2.x(2)
sv_latex_from_qc(qc_ej2, backend)

<IPython.core.display.Latex object>

In [46]:
qc_ej2 = QuantumCircuit(4, 4)
qc_ej2.x(3)
sv_latex_from_qc(qc_ej2, backend)

<IPython.core.display.Latex object>

### <span style="color:red;">JHZ: Mi versión en base a lo del vídeo de complementos formativos </span>

# Ejercicio 2
Implementar los circuitos de cuatro cúbits que suman al registro cuántico los valores 1, 2, 3, 4, 5, 6, 7 y 8 y verificar que funcionan correctamente. Utilizar QISKit Quantum lab.

Para simplificar las pruebas, implementaremos una sencilla función que encapsula la ejecución del circuito cuántico y la representación de su vector de estado de salida.

In [47]:
def sv_latex_from_qc(qc, backend):
    sv = backend.run(qc).result().get_statevector()
    return sv.draw(output='latex')

Puesto que la generación de circuitos de adición binaria es conceptualmente sencilla de automatizar, en lugar de crear los circuitos de suma pedidos en el enunciado vamos a definir una función que los cree en base a un parámetro (el valor a sumar). Veamos la implementación de esta función:

### `circuit_adder` Function:

**Purpose:**
The `circuit_adder` function is designed to create a quantum circuit to add a number to an existing 4-qubit circuit.

**Attributes:**
- `num` (int): A number to add to the circuit. It must be between 0 and 8.

**Methods:**
None

**Example Usage:**
```python
qc_adder_7 = circuit_adder(7)
qc_final = qc_already_exist.compose(qc_adder_7) # Concatenate both quantum circuits to add 7 to the already existing qc_already_exist quantum circuit


In [55]:
def circuit_adder (num):
    if num<1 or num>8:
        raise ValueError("Out of range")  ## El enunciado limita el sumador a los valores entre 1 y 8. Quitar esta restricción sería directo.
    # Definición del circuito base que vamos a construir
    qreg_q = QuantumRegister(4, 'q')
    creg_c = ClassicalRegister(1, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)
    
    qbit_position = 0
    for element in reversed(np.binary_repr(num)):
        if (element=='1'):
            circuit.barrier()
            match qbit_position:
                case 0: # +1
                    circuit.append(C3XGate(), [qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3]])
                    circuit.ccx(qreg_q[0], qreg_q[1], qreg_q[2])
                    circuit.cx(qreg_q[0], qreg_q[1])
                    circuit.x(qreg_q[0])
                case 1: # +2
                    circuit.ccx(qreg_q[1], qreg_q[2], qreg_q[3])
                    circuit.cx(qreg_q[1], qreg_q[2])
                    circuit.x(qreg_q[1])
                case 2: # +4
                    circuit.cx(qreg_q[2], qreg_q[3])
                    circuit.x(qreg_q[2])
                case 3: # +8
                    circuit.x(qreg_q[3])
        qbit_position+=1
    return circuit

In [49]:
def circuit_adder(num):
    if not 1 <= num <= 8:
        raise ValueError("Out of range")

    qreg_q = QuantumRegister(4, 'q')
    creg_c = ClassicalRegister(1, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)

    for qbit_position, bit_value in enumerate(reversed(np.binary_repr(num))):
        if bit_value == '1':
            circuit.barrier()
            if qbit_position == 0:  # +1
                circuit.mcx(qreg_q[:3], qreg_q[3])
                circuit.x(qreg_q[0])
            else:  # +2, +4, +8
                circuit.mcx(qreg_q[qbit_position:3], qreg_q[3])
                circuit.x(qreg_q[qbit_position])

    return circuit

Probamos la función generadora de circuitos generando un sumador para el número binario 3

In [50]:
add_3 = circuit_adder(3)
add_3.draw()

░      ┌───┐ ░           
q_0: ─░───■──┤ X ├─░───────────
      ░   │  └───┘ ░      ┌───┐
q_1: ─░───■────────░───■──┤ X ├
      ░   │        ░   │  └───┘
q_2: ─░───■────────░───■───────
      ░ ┌─┴─┐      ░ ┌─┴─┐     
q_3: ─░─┤ X ├──────░─┤ X ├─────
      ░ └───┘      ░ └───┘     
c: 1/══════════════════════════

In [51]:
add_3 = circuit_adder(3)
add_3.draw()

░      ┌───┐ ░           
q_0: ─░───■──┤ X ├─░───────────
      ░   │  └───┘ ░      ┌───┐
q_1: ─░───■────────░───■──┤ X ├
      ░   │        ░   │  └───┘
q_2: ─░───■────────░───■───────
      ░ ┌─┴─┐      ░ ┌─┴─┐     
q_3: ─░─┤ X ├──────░─┤ X ├─────
      ░ └───┘      ░ └───┘     
c: 1/══════════════════════════

A continuación, reutilizaremos el circuito anterior (+3) y creamos un circuito cuántico que represente el número 2 para validar que genera un valor equivalente a 5.

In [56]:
qc_test_2 = QuantumCircuit(4, 4)
qc_test_2.x(1)

qc_test_2_plus_3 = qc_test_2.compose(add_3)
qc_test_2_plus_3.draw()

░      ┌───┐ ░           
q_0: ──────░───■──┤ X ├─░───────────
     ┌───┐ ░   │  └───┘ ░      ┌───┐
q_1: ┤ X ├─░───■────────░───■──┤ X ├
     └───┘ ░   │        ░   │  └───┘
q_2: ──────░───■────────░───■───────
           ░ ┌─┴─┐      ░ ┌─┴─┐     
q_3: ──────░─┤ X ├──────░─┤ X ├─────
           ░ └───┘      ░ └───┘     
c: 4/═══════════════════════════════

Y validamos su resultado:

In [57]:
sv_latex_from_qc(qc_test_2_plus_3, backend)

<IPython.core.display.Latex object>

Realizamos otra prueba sumando 8 a un estado previo de 7:

In [58]:
qc_test_7 = QuantumCircuit(4, 4)
qc_test_7.x(0)
qc_test_7.x(1)
qc_test_7.x(2)

qc_test_7_plus_8 = qc_test_7.compose(circuit_adder(8))
sv_latex_from_qc(qc_test_7_plus_8, backend)
#qc_test_7_plus_8.draw()

<IPython.core.display.Latex object>

Con lo que validamos que se realizan todas las sumas correctamente, incluyendo aquellas que requieren de acarreo.

# Ejercicio 3
Implementar en Python el algoritmo de teleportación cuántica para teleportar el estado $|{\Psi}\rangle$ del cúbit de Alice al cúbit de Bob. El estado $|{\Psi}\rangle$ viene determinado por los ángulos $\theta=37.5$ grados y $\phi=13.4$ grados, de la esfera de Bloch.

En primer lugar convertiremos los ángulos proporcionados por el enunciado a radianes, que nos permitan operar correctamente en el sistema:

* $\theta=37.5$ grados = 6.544985 radianes
* $\phi=13.4$ grados = 2.338741 radianes

Utilizaremos $\theta$ como ángulo de giro en el eje X del qubit y $\phi$ sobre el eje Z.

In [63]:
theta = 6.544985
phi = 2.338741
lmbda = 0

Para realizar este algoritmo, Alice dispone de dos qubits y Bob dispone de uno. En el circuito, los dos primeros qubits serán los de Alice y el tercero será de Bob

In [61]:
alice_1 = 0
alice_2 = 1
bob_1 = 2

Creamos un circuito cuántico con 3 cúbits e inicializamos el qubit 1 de Alice con los ángulos proporcionados, donde theta, phi, y lmbda son los 3 ángulos de Euler.

In [ ]:
qc = QuantumCircuit(3, 3)
qc.u(theta, phi, lmbda, alice_1)

En el algoritmo de teletransportación cuántica, la creación de un par entrelazado se realiza típicamente utilizando un estado Bell específico. La elección del estado Bell es crucial para el éxito del protocolo de teletransportación. La elección estándar es el estado Bell maximalmente entrelazado 
$\frac{|00\rangle + |11\rangle}{\sqrt{2}}$.

In [ ]:
qc.h(alice_2)
qc.cx(alice_2, bob_1)

Alice entrelaza su qubit con el estado cuántico

In [ ]:
qc.cx(alice_1, alice_2)
qc.h(alice_1)

Alice mide sus qubits y envía los resultados a Bob.

In [ ]:
qc.measure([alice_1, alice_2], [alice_1, alice_2])

In [64]:
qreg_q = QuantumRegister(3, 'q')
creg_c = ClassicalRegister(2, 'c')
qc_ej3 = QuantumCircuit(qreg_q, creg_c)

# Creación del estado Psi
qc_ej3.rx(theta, alice_1)
qc_ej3.rz(phi, alice_1)
qc_ej3.barrier()

# Implementación de la primera parte del protocolo de teleportación
qc_ej3.h(alice_2)
qc_ej3.cx(alice_2, bob_1) # Entrelazamiento del qubit para Bob
qc_ej3.cx(alice_1, alice_2)
qc_ej3.h(alice_1)
qc_ej3.barrier()

# Mediciones de Alice
qc_ej3.measure(alice_1, 0)
qc_ej3.measure(alice_2, 1)


qc_ej3.draw()


┌───────────┐┌────────────┐ ░                ┌───┐ ░ ┌─┐   
q_0: ┤ Rx(6.545) ├┤ Rz(2.3387) ├─░─────────────■──┤ H ├─░─┤M├───
     └───────────┘└────────────┘ ░ ┌───┐     ┌─┴─┐└───┘ ░ └╥┘┌─┐
q_1: ────────────────────────────░─┤ H ├──■──┤ X ├──────░──╫─┤M├
                                 ░ └───┘┌─┴─┐└───┘      ░  ║ └╥┘
q_2: ────────────────────────────░──────┤ X ├───────────░──╫──╫─
                                 ░      └───┘           ░  ║  ║ 
c: 2/══════════════════════════════════════════════════════╩══╩═
                                                           0  1

<span style="color:red;">Seguir con el protocolo: Alice mide y c0,c1 y en base a eso Bob modifica q2 siguiendo el algoritmo correspondiente. </span>